Se importan las bibliotecas necesarias para el análisis de datos, manipulación de datos, y creación de modelos estadísticos.

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from statsmodels.miscmodels.ordinal_model import OrderedModel
import statsmodels.api as sm
from scipy.stats import chi2
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm

Se carga el archivo de Excel en un DataFrame de pandas.

In [38]:
df = pd.read_excel('Databases/1. Generales - Original.xlsx')

Se renombra una columna para evitar problemas con el nombre.

In [39]:
df = df.rename(columns={'autopercep_izq-der': 'autopercep_izq_der'})

Se definen las variables dependientes (la variable que se quiere predecir) y las variables independientes (las que influyen en la dependiente). Al ser un modelo ordinal, la variable dependiente debe ser ordinal, o sea, debe tener categorías que pueden ser ordenadas jerárquicamente, como calificaciones (por ejemplo, "mala", "regular", "buena") o niveles de satisfacción (por ejemplo, "insatisfecho", "neutral", "satisfecho") 

In [40]:
# Variables a explicar.
Variables_Dependientes = ['cercania_Massa', 
                          #'cercania_Milei', 
                          #'cercania_Bregman', 
                          #'cercania_Bullrich', 
                          #'cercania_Schiaretti'
                          ]

# Variables que influyen en las otras.
Variables_Todas = [
                            # 'genero',
                            # 'edad',
                            # 'nacionalidad',
                            # 'provincia',
                            'e_social',
                            # 'niv_educativo',
                            'voto_2019',
                            # 'voto_PASO_2023',
                            'candidato_PASO_2023',
                            # 'autopercep_izq_der',
                            # 'autopercep_conpro',
                            # 'autopercep_perantiper',
                            'indice_positividad',
                            'massa_ip_izqder',
                            'massa_ip_conpro',
                            # 'bullrich_ip_izqder',
                            # 'bullrich_ip_conpro',
                            # 'schiaretti_ip_izqder',
                            # 'schiaretti_ip_conpro',
                            # 'milei_ip_izqder',
                            # 'milei_ip_conpro',
                            # 'bregman_ip_izqder',
                            # 'bregman_ip_conpro',
                            # 'indice_progresismo',
                            # 'indice_conservadurismo'
                            ]

Se crea una lista de variables que se utilizarán para el análisis de un candidato específico (Massa).

In [41]:
Variables_Massa = [
    'voto_2019',
    'autopercep_perantiper',
    'massa_ip_conpro',
    'indice_conservadurismo',
    'massa_ip_izqder',
    'edad',
    'autopercep_conpro',
    'indice_progresismo',
    'autopercep_izq_der',
    'indice_positividad',
    'candidato_PASO_2023',
    'schiaretti_ip_conpro',
    'schiaretti_ip_izqder'
]

In [42]:
Variables_Independientes = Variables_Massa

Se asignan las variables independientes a X y la variable dependiente a y.

In [43]:
X = df[Variables_Independientes]
y = df[Variables_Dependientes]

Se convierte cualquier variable categórica en variables numéricas usando LabelEncoder. Esto es necesario porque muchos modelos estadísticos requieren entradas numéricas.

In [44]:
# Convert categorical independent variables to numerical using LabelEncoder.
for Column in Variables_Independientes:
    if X[Column].dtype == 'object':
        X[Column] = LabelEncoder().fit_transform(X[Column])

C:\Users\tomas\AppData\Local\Temp\ipykernel_14012\3576202961.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[Column] = LabelEncoder().fit_transform(X[Column])
C:\Users\tomas\AppData\Local\Temp\ipykernel_14012\3576202961.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[Column] = LabelEncoder().fit_transform(X[Column])


Los datos se dividen en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%) para poder evaluar el modelo posteriormente.

In [45]:
# Split the data into training and testing sets.
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, y, test_size=0.2, random_state=42)

Se ajusta un modelo de regresión logística ordinal a los datos de entrenamiento. Esto es apropiado ya que la variable dependiente es ordinal.

In [46]:
# Ajustar el modelo de regresión logística ordinal sin añadir la constante manualmente.
Ordinal_Model = OrderedModel(Y_Train, X_Train, distr='logit')
Ordinal_Result = Ordinal_Model.fit()

C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\optimizer.py:737: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,
C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Se imprime un resumen del modelo ajustado para revisar los coeficientes y la significancia estadística.

In [47]:
# Resumen del modelo ajustado para verificar los coeficientes de los términos.
print(Ordinal_Result.summary())

                             OrderedModel Results                             
Dep. Variable:         cercania_Massa   Log-Likelihood:                -2892.7
Model:                   OrderedModel   AIC:                             5819.
Method:            Maximum Likelihood   BIC:                             5917.
Date:                Mon, 21 Oct 2024                                         
Time:                        12:32:03                                         
No. Observations:                2271                                         
Df Residuals:                    2254                                         
Df Model:                          13                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
voto_2019                 -0.1931      0.020     -9.882      0.000      -0.231      -0.155
autopercep_peran

Se realizan predicciones sobre el conjunto de prueba utilizando el modelo ajustado.

In [48]:
# Realizar predicciones en el set de prueba.
Predictions = Ordinal_Result.predict(X_Test)

__1. Verificar la linealidad del logit.__

----

Para verificar la linealidad entre las variables independientes y el logit, se pueden agregar términos cuadráticos y observar si estos son significativos. Verificá si el término cuadrático (edad_sq) es significativo (p-valor < 0.05). Si es significativo, puede indicar que la relación no es lineal.

In [49]:
# # Verificar la linealidad del logit para cada variable independiente.
# for Variable in Variables_Independientes:

#     # Agregar el término cuadrático.
#     X_Train[f'{Variable}_sq'] = X_Train[Variable] ** 2  

#     # Ajustar el modelo ordinal con el término cuadrático.
#     Ordinal_Model = OrderedModel(Y_Train, X_Train[[Variable, f'{Variable}_sq']], distr='logit')
#     Ordinal_Result = Ordinal_Model.fit(disp=0)  # 'disp=0' oculta la salida durante el ajuste.

#     # Extraer el p-valor del término cuadrático.
#     p_value = Ordinal_Result.pvalues[f'{Variable}_sq']

#     # Imprimir solo el p-valor.
#     print(f"P-value for {Variable} (quadratic term): {p_value:.4f}")  # Format p-value to 4 decimal places.

__2. Verificar la proporcionalidad de las odds.__

-----

Se puede usar el test de Brant o el test de proporcionalidad de odds. El test de Brant evalúa si las pendientes son constantes entre las categorías de la variable dependiente. Por lo tanto, se necesita ajustar un modelo para cada nivel de la variable dependiente y comparar los coeficientes entre modelos.

In [53]:
def Brant_Test(Ordinal_Model_Result: sm.regression.linear_model.RegressionResultsWrapper, 
               Y_Train: pd.Series, X_Train: pd.DataFrame) -> pd.DataFrame:
    
    """
    Performs the Brant test for proportional odds assumption.
    
    Parameters:
    Ordinal_Model_Result (sm.regression.linear_model.RegressionResultsWrapper): The fitted ordinal model result.
    Y_Train (pd.Series): The dependent ordinal variable.
    X_Train (pd.DataFrame): The independent variables.
    
    Returns:
    pd.DataFrame: A DataFrame with chi-square statistics and p-values for each independent variable.
    
    """
    
    # Ensure Y_Train is a Series if it's a DataFrame.
    if isinstance(Y_Train, pd.DataFrame):
        Y_Train = Y_Train.iloc[:, 0]  # Convert Y_Train to a Series.
    
    # Get the number of categories in the dependent variable.
    Categories = np.sort(Y_Train.unique())
    K_Categories = len(Categories)
    
    # Create an empty DataFrame to store the results.
    Results = pd.DataFrame(columns=['Variable', 'Chi2', 'P_Value'])
    
    # For each variable in the model, compare the coefficients across categories.
    for Variable in X_Train.columns:
        Chi2_Stat = 0
        Df = 0
        
        # Fit separate binary models for each threshold in the ordinal outcome.
        for Index in range(K_Categories - 1):
            Y_Binary = (Y_Train > Categories[Index]).astype(int)
            Binary_Model = sm.Logit(Y_Binary, X_Train[Variable])
            Binary_Result = Binary_Model.fit(disp=0)
            
            # Check if the variable exists in both models.
            if Variable in Ordinal_Model_Result.params.index and Variable in Binary_Result.params.index:
                # Extract coefficients for the current variable.
                Beta_Diff = Ordinal_Model_Result.params[Variable] - Binary_Result.params[Variable]
                Chi2_Stat += Beta_Diff ** 2 / Ordinal_Model_Result.bse[Variable] ** 2
                Df += 1
            else:
                print(f"Warning: Variable '{Variable}' not found in one of the models.")
        
        # Calculate the p-value from the chi-square distribution.
        P_Value = chi2.sf(Chi2_Stat, Df)
        
        # Append the results to the DataFrame using pd.concat.
        New_Row = pd.DataFrame({'Variable': [Variable], 'Chi2': [Chi2_Stat], 'P_Value': [P_Value]})
        Results = pd.concat([Results, New_Row], ignore_index=True)
    
    return Results

La hipótesis nula del test de Brant es que los coeficientes son proporcionales para todas las categorías de la variable dependiente. Un p-valor bajo (menor que 0.05) indica que la hipótesis nula debe rechazarse, lo que significa que la suposición de odds proporcionales no se sostiene para esa variable.

In [55]:
Brant_Results = Brant_Test(Ordinal_Result, Y_Train, X_Train)
print(Brant_Results)

                  Variable         Chi2        P_Value
0                voto_2019  1592.797717   0.000000e+00
1    autopercep_perantiper   650.307909  1.999192e-139
2          massa_ip_conpro   342.696398   6.619563e-73
3   indice_conservadurismo    26.334211   2.709442e-05
4          massa_ip_izqder   339.724171   2.900543e-72
5                     edad   208.304729   6.152079e-44
6        autopercep_conpro   551.564607  4.692515e-118
7       indice_progresismo   171.742620   4.420184e-36
8       autopercep_izq_der   727.485338  3.895405e-156
9       indice_positividad   159.029853   2.360383e-33
10     candidato_PASO_2023  2250.339751   0.000000e+00
11    schiaretti_ip_conpro   667.811331  3.247390e-143
12    schiaretti_ip_izqder   203.573715   6.404231e-43


C:\Users\tomas\AppData\Local\Temp\ipykernel_14012\3710477143.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Results = pd.concat([Results, New_Row], ignore_index=True)


__3. Independencia de las observaciones.__

------

Para verificar la independencia de las observaciones, se puede revisar si hay duplicados o agrupaciones en los datos. En el contexto de encuestas, esto suele ser asumido, pero es bueno verificar:

In [143]:
# Revisar duplicados en el DataFrame
print(df.duplicated().sum())

0
